# Notes

code for producing GOIT pipelines summary stats, and for calculating landing page stats

this is saved as an Excel file, which Baird copies/pastes into the existing summary tables information on the drive here:
https://docs.google.com/spreadsheets/d/1OYH6D7c-D0FsL5GzBGijtkmvQCTkBUclj-UVoOieUFo/edit

In [386]:
import pandas
import numpy
import pygsheets
import datetime
import re
import pytz

In [387]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")

## import data

In [388]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
spreadsheet = gc.open_by_key('1WaBMIdfRWqSqXUw7_cKXo3RipyhPdnNN8flqEYfMZIA') # file to use for gas pipelines Dec 2023
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # CURRENT sheet

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A3')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A3')

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')

country_ratios_df = country_ratios_df.loc[country_ratios_df.Wiki!='']

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['Wiki']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig.Fuel==fuel_type]

owners_df_orig = owners_df_orig.loc[owners_df_orig['ProjectID']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig['Wiki']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig.Status!='N/A']

owners_df_orig.set_index('ProjectID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parent_metadata_df.set_index('Parent', inplace=True)

In [389]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

In [390]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')

#region_name = 'Global'; region_df_touse = region_df_orig.copy()
#region_name = 'AsiaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AsiaGasTracker=='Yes']
#region_name = 'EuroGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.EuroGasTracker=='Yes']
region_name = 'AfricaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AfricaGasTracker=='Yes']
#region_df_agt.copy()

#region_df_touse = region_df_orig.copy()

In [391]:
region_df_touse_cleaned = region_df_touse.loc[(region_df_touse.Region!='--')&
                                            (region_df_touse.SubRegion!='--')]
multiindex_region_subregion = region_df_touse_cleaned.groupby(['Region','SubRegion'])['Country'].count().index
multiindex_region_subregion

MultiIndex([('Africa',    'Northern Africa'),
            ('Africa', 'Sub-Saharan Africa')],
           names=['Region', 'SubRegion'])

## file names with regional specifics

In [392]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

if fuel_type=='Gas':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-gas-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='NGL':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-NGL-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='Oil':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-Oil-pipelines-'+str(datetime.date.today())+'.xlsx')

### create country-specific dataframes for region, country_ratios_df, owners_df

In [393]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

In [394]:
pipes_df_touse.head()

,PipelineNetworkGrouping,PipelineName,SegmentName,Wiki,ProjectID,Researcher,LastUpdated,Fuel,PipelineType,Countries,...,CostUSDPerKm,CostEuroPerKm,AssociatedWithLNGExports,AssociatedLNGTerminal,DraftPCI6List,PCI6ProjectCode,PipelineDirectionality,QCCOwner,QCCOwner2024Update,ImpactedByRussiaUkraineInvasion
221,,Arab Gas Pipeline,,https://www.gem.wiki/Arab_Gas_Pipeline,P0436,NA,2022-07-14,Gas,,"Egypt, Jordan, Syria, Lebanon",...,1000000.00,9.174312e+05,,,,,,,,
223,,Escravos-Lagos Pipeline System (ELPS),,https://www.gem.wiki/Escravos%E2%80%93Lagos_Pi...,P0438,CJ,23/08/2023,Gas,,Nigeria,...,NaN,NaN,,,,,,,,
224,,Greenstream Gas Pipeline,,https://www.gem.wiki/Greenstream_Pipeline,P0439,NA,2022-07-20,Gas,,"Libya, Italy",...,12692307.69,1.164432e+07,,,,,,,,
238,,Mozambique-South Africa Gas Pipeline,,https://www.gem.wiki/Mozambique-South_Africa_G...,P0455,CJ,2023-08-22,Gas,,"Mozambique, South Africa",...,NaN,NaN,,,,,,,,
239,,Mtwara–Dar es Salaam Gas Pipeline,,https://www.gem.wiki/Mtwara%E2%80%93Dar_es_Sal...,P0456,CJ,2023-08-15,Gas,transmission,Tanzania,...,NaN,NaN,,,,,,,,


### sum MergedKmByCountry and MergedKmByRegion

In [395]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [396]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']
excel_status_list_with_countries = ['Country']+excel_status_list

In [397]:
country_ratios_df_subset = country_ratios_df_touse.loc[country_ratios_df_touse['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby(['Region','SubRegion'])['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.names = ['Region','Subregion']
km_by_country.index.name = 'Country'

km_by_region.loc['Total',:] = km_by_region.sum(axis=0).values
km_by_country.loc['Total',:] = km_by_country.sum(axis=0).values

# drop all-zero rows
km_by_country = km_by_country.loc[~(km_by_country==0).all(axis=1)]

km_by_country.replace(0,'',inplace=True)
km_by_region.replace(0,'',inplace=True)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


In [398]:
km_by_region

Proposed  Construction  \
Region Subregion                                    
Africa Northern Africa      4068.97        188.25   
       Sub-Saharan Africa  18610.27       1582.00   
Total                      22679.24       1770.25   

                           In Development (Proposed + Construction)  Shelved  \
Region Subregion                                                               
Africa Northern Africa                                      4257.22   172.20   
       Sub-Saharan Africa                                  20192.27  1319.72   
Total                                                      24449.49  1491.92   

                           Cancelled  Operating Idle Mothballed Retired  
Region Subregion                                                         
Africa Northern Africa        589.00   24853.55          1107.0          
       Sub-Saharan Africa     228.96    7139.72                          
Total                         817.96   31993.27          1107.0

## pipeline km by parent company (owner) and project status

### first check that there are no missing projectids

In [399]:
owner_parent_calculations_df = pandas.DataFrame()
# needs country, km in each country columns as well

for idx,row in country_ratios_df_subset.iterrows():
    #print(row.ProjectID)
    parent_string = row.Parent
    #print(parent_string)

    #print(parent_string)
    # if the first letter of the parent_string is a chinese character, and it ends with [100.00%],
    # that means it's a non-researched QCC owner; keep as-is
    if re.findall(r'[\u4e00-\u9fff]+', parent_string[:1]) != [] and parent_string[-9:]=='[100.00%]':
        parent_list = [parent_string.split(' [100.00%]')[0]]
        percent_list = [1.0]
    # otherwise do the rest of the thing
    else:
        parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
        percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        #print(parent_list)
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'ParentHQCountry':parent_metadata_df.loc[parent,'ParentHQCountry'],
                                                                         'ParentHQRegion':parent_metadata_df.loc[parent,'ParentHQRegion'],
                                                                         'Country':row.Country,
                                                                         'Status':row.Status,
                                                                         'MergedKmByCountry':row.MergedKmByCountry}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df.FractionOwnership*owner_parent_calculations_df.MergedKmByCountry

In [400]:
owner_parent_calculations_df

,Parent,ProjectID,FractionOwnership,ParentHQCountry,ParentHQRegion,Country,Status,MergedKmByCountry,KmOwnership
0,Moroccan National Board of Hydrocarbons and Mines,P0463,0.500,Morocco,Africa,Joint regime area (Senegal/Guinea Bissau),Proposed,71.27,35.635
0,Nigerian National Petroleum Corporation,P0463,0.500,Nigeria,Africa,Joint regime area (Senegal/Guinea Bissau),Proposed,71.27,35.635
0,ExxonMobil,P1076,1.000,United States,Americas,Papua New Guinea,Cancelled,11.00,11.000
0,Santos Limited,P1078,0.425,Australia,Oceania,Papua New Guinea,Operating,716.00,304.300
0,ExxonMobil,P1078,0.332,United States,Americas,Papua New Guinea,Operating,716.00,237.712
...,...,...,...,...,...,...,...,...,...
0,Nigerian National Petroleum Corporation,P4185,1.000,Nigeria,Africa,Nigeria,Proposed,686.00,686.000
0,"Genser Power USA, LLC",P4214,1.000,United States,Americas,Ghana,Operating,80.00,80.000
0,"Genser Power USA, LLC",P4215,1.000,United States,Americas,Ghana,Construction,155.00,155.000
0,"Genser Power USA, LLC",P4216,1.000,United States,Americas,Ghana,Proposed,85.00,85.000


In [401]:
#unique_owner_list = owner_parent_calculations_df.Parent.sort_values().unique().tolist()

##################################################
# create km count by owner, status
##################################################
owners_km_by_status_df = \
    owner_parent_calculations_df.groupby(
    ['Parent','ParentHQCountry','Status'])[['KmOwnership']].sum().unstack().droplevel(axis=1, level=[0])

owners_km_by_status_df = owners_km_by_status_df.reindex(columns=status_list)
owners_km_by_status_df = owners_km_by_status_df.reset_index().set_index('Parent')
owners_km_by_status_df.columns.name = None

owners_km_by_status_df['In Development (Proposed + Construction)'] = owners_km_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_km_by_status_df = owners_km_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list_with_countries]

# totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_km_by_status_df.loc['Total',:] = owners_km_by_status_df.sum(axis=0, min_count=0).values
owners_km_by_status_df.loc['Total','Country'] = ''

owners_km_by_status_df = owners_km_by_status_df.replace(numpy.nan, '')
owners_km_by_status_df = owners_km_by_status_df.replace(0, '')

owners_km_by_status_df.to_excel(excel_writer, sheet_name='Kilometers by owner')

In [402]:
owners_km_by_status_df

,Country,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Parent,,,,,,,,,,
APA Corporation,United States,,,,,,369.5,,,
Atlas Petroleum International,Nigeria,,,,,,14.0,,,
BASF,Germany,,,,,,129.0,,,
BP,United Kingdom,,,,,,275.145,,,
BPCL Group,India,,,,4.5,,,,,
...,...,...,...,...,...,...,...,...,...,...
Transnet,South Africa,3660.02,,3660.02,,,600.0,,,
Tunisian Company of Electricity and Gas,Tunisia,,,,,,294.0,,,
other,unknown,,,,34.42278,,,,,


### pipeline km by start year, type

In [403]:
pipes_started = pipes_df_touse.copy()
#pipes_started['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='Gas')]
if fuel_type == 'Oil':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='Oil')]
if fuel_type == 'NGL':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='NGL')]

pipes_started_sum = pipes_started.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [404]:
if fuel_type == 'Gas':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['Gas pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Gas pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'Oil':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['Oil pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Oil pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'NGL':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['NGL pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['NGL pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

km_by_start_year.loc['Total',:] = km_by_start_year.sum(axis=0)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')
#km_by_start_year

## save excel file

In [405]:
excel_writer.close()

## calculating stats for landing page

In [406]:
# number of projects tracked in total
print(pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type].shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type]['LengthMergedKm'].sum()/1e6, 'M km tracked')

161 gas pipeline projects tracked
0.06936539 M km tracked


In [407]:
# number of projects tracked in total
print(pipes_df_touse.loc[(pipes_df_touse.Fuel==fuel_type)&
                        (pipes_df_touse.Status.isin(['Proposed','Construction']))].shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse.loc[(pipes_df_touse.Fuel==fuel_type)&
                        (pipes_df_touse.Status.isin(['Proposed','Construction']))]['LengthMergedKm'].sum()/1e3, 'K km tracked')

29 gas pipeline projects tracked
25.065599999999996 K km tracked
